# Final Submission - Code
## Submission by:
- **Name**: Vignesh Subramanian
- **GT ID**: 903947262

### Choose 'verbose' which should be a multiple of number 100 (1, 2, 5, 10, 20 , 50, 100) to assert the frequency of the output logs. All the outputs of the sat solvers are not shown in the output block but is saved a text file in the relative folder. You can open that to see the sat solving process in realtime.

### Press 'RUN ALL' on the menu panel or run all the cells in order to get the required results (Dictionaries stored in form of text file (.txt)) for each heuristic. Allow all the cells to finish running and then run the 'plots.ipynb' file to get all the necessary plot. WARNING: Unless this file has completely finished its run, the 'plots.ipynb' file will not be able to generate the plots.

In [1]:
# Import Libraries and Set Seed 

import random
from collections import OrderedDict
import time
import json
import multiprocessing
import sys
import statistics

verbose = 1 
call_counter_rh = 0
call_counter_tch = 0
call_counter_jwh = 0

random.seed(1235)

In [2]:
# This function creates a dictionary that maps each number from 1 to N to a unique variable name.
def generate_mapping(N):
    return {i: f'var{i}' for i in range(1, N+1)}

# This function generates a random 3-SAT problem consisting of N variables and L clauses.
def generate_random_3sat(N, L):
    # Ensure there are at least 3 variables for 3-SAT problems.
    if N < 3:
        raise ValueError("The number of variables (N) should be at least 3 for 3-SAT problems.")
    # Ensure there is at least 1 clause.
    if L < 1:
        raise ValueError("The number of clauses (L) should be at least 1.")

    # Generate a mapping for the given number of variables.
    mapping = generate_mapping(N)
    clauses = []
    
    # For each clause, randomly select 3 literals and determine their negation status.
    for _ in range(L):
        literals = random.sample(range(1, N+1), 3)
        literals = [('¬' + mapping[lit] if random.random() < 0.5 else mapping[lit]) for lit in literals]
        clauses.append(literals)
    
    return clauses, mapping

# This function generates 100 random 3-SAT formulas for each specified L/N ratio.
def generate_formulas_based_on_ratios(N):
    # Generate L/N ratios from 3 to 6 at increments of 0.2.
    ratios = [i * 0.2 + 3 for i in range(16)]
    
    formulas = {}
    
    # For each ratio, compute the number of clauses (L) and generate 100 formulas.
    for ratio in ratios:
        L = int(ratio * N)
        formulas_for_ratio = []

        for _ in range(100):
            clauses, mapping = generate_random_3sat(N, L)
            formulas_for_ratio.append(clauses)

        formulas[ratio] = formulas_for_ratio

    return formulas
    return formulas

In [3]:
N = 150  # No. of Variables
formulas = generate_formulas_based_on_ratios(N)
with open("formula_150.json", "w") as file:
    json.dump(formulas, file)

## Random Choice Heurisitic

In [4]:
# Function to perform Pure Literal Elimination
def pure_literal_elimination(clauses):
    assignment = {}  # Dictionary to store variable assignments
    # Flatten the list of clauses to get all literals
    literals = [lit for clause in clauses for lit in clause]
  
    # Loop through all literals to find pure literals and assign them
    for literal in literals:
        if literal.startswith('¬'):
            pos_literal = literal[1:]
            # If the positive form of the literal is not in literals, it's a pure literal
            if pos_literal not in literals:
                assignment[pos_literal] = False  # Assign False to the variable
        else:
            neg_literal = '¬' + literal
            # If the negative form of the literal is not in literals, it's a pure literal
            if neg_literal not in literals:
                assignment[literal] = True  # Assign True to the variable

    return assignment

# Function to perform Unit Propagation
def unit_propagation(clauses):
    assignment = {}  # Dictionary to store variable assignments
    # Find unit clauses (clauses with only one literal)
    unit_clauses = [clause for clause in clauses if len(clause) == 1]

    # Assign values to make all unit clauses true
    for clause in unit_clauses:
        literal = clause[0]
        if literal.startswith('¬'):
            assignment[literal[1:]] = False  # Negate the literal and assign False
        else:
            assignment[literal] = True  # Assign True to the literal

    return assignment

# Function to apply the current assignment to clauses
def apply_assignment(clauses, assignment):
    updated_clauses = []  # List to store updated clauses

    # Loop through all clauses
    for clause in clauses:
        new_clause = []  # List to store literals that are not yet assigned
        for literal in clause:
            var = literal[1:] if literal.startswith('¬') else literal  # Remove the negation symbol if it exists
            if var in assignment:
                # If the literal makes the clause true, skip adding the clause
                if (literal.startswith('¬') and not assignment[var]) or \
                   (not literal.startswith('¬') and assignment[var]):
                    new_clause = None
                    break
            else:
                new_clause.append(literal)  # Add unassigned literals to new clause
        
        if new_clause is not None:
            updated_clauses.append(new_clause)  # Add the updated clause to the list of clauses

    return updated_clauses

# DPLL algorithm
def dpll_rh(clauses, assignment):
    # Apply unit propagation and pure literal elimination to simplify clauses and assignment
    new_assignment = {**unit_propagation(clauses), **pure_literal_elimination(clauses)}
    if new_assignment:
        assignment.update(new_assignment)
        clauses = apply_assignment(clauses, assignment)

    # If there is an empty clause, the formula is unsatisfiable
    if any(len(clause) == 0 for clause in clauses):
        return False, {}

    # If there are no more clauses, the formula is satisfiable with the current assignment
    if not clauses:
        return True, assignment

    # Choose an unassigned variable randomly
    unassigned = list(set(var.replace('¬', '') for clause in clauses for var in clause) - set(assignment.keys()))
    if not unassigned:
        return True, assignment
    
    # Random Choice Heuristic
    literal = random.choice(unassigned)

    global call_counter_rh
    call_counter_rh += 1

    # Try to assign True to the chosen variable and solve recursively
    new_clauses = apply_assignment(clauses, {literal: True})
    result, new_assignment = dpll_rh(new_clauses, {**assignment, literal: True})
    if result:
        return True, new_assignment

    # Try to assign False to the chosen variable and solve recursively
    new_clauses = apply_assignment(clauses, {literal: False})
    result, new_assignment = dpll_rh(new_clauses, {**assignment, literal: False})
    if result:
        return True, new_assignment

    # If neither True nor False leads to a solution, the formula is unsatisfiable
    return False, {}

### Solver Cell

In [5]:
results = {}
TIME_THRESHOLD = 360  # seconds

original_stdout = sys.stdout 

def run_dpll(clauses, shared_result, shared_call_counter):
    global call_counter_rh
    call_counter_rh = 0
    result, assignment = dpll_rh(clauses, {})
    shared_result.put(result)
    shared_call_counter.put(call_counter_rh)

def print_elapsed_time(start_time, end_signal):
    while not end_signal.is_set():
        elapsed_time = time.time() - start_time.value
        print(f"Time elapsed: {elapsed_time:.2f} seconds")
        time.sleep(1)

with open('output_log_rh.txt', 'w') as f:
    sys.stdout = f

    for ratio, formula_list in formulas.items():

        print('Verifying for Random Choice Heuristic - Ratio: {}\n'.format (round(ratio, 1)))
        print('No of variables (N):        ', N)
        print('No of clauses (L):          ', int(N * ratio))
        print('------------------------------------------\n')
        print('Logs: \n')

        satisfiable_count = 0
        run_time = []
        dpll_counter = []
        formula_counter = 0
        result_list = []
        termination_check = False
        unsolved_counter = 0

        for clauses in formula_list:

            formula_counter += 1
            termination_check = False

            if(formula_counter % verbose == 0):
                print('Checking satisfiability for formula {}...'.format(formula_counter))

            shared_result = multiprocessing.Queue()
            shared_call_counter = multiprocessing.Queue()
            start_time = multiprocessing.Value('d', time.time())
            end_signal = multiprocessing.Event()

            process = multiprocessing.Process(target=run_dpll, args=(clauses, shared_result, shared_call_counter))
            
            process.start()
            process.join(TIME_THRESHOLD)
            
            end_signal.set()  
            
            if process.is_alive():
                print("Terminating satisfiability check for formula {} due to time exceeding threshold".format(formula_counter))
                termination_check = True
                process.terminate()
                process.join()


            end_time = time.time()
            elapsed_time = end_time - start_time.value
            run_time.append(elapsed_time)
            
            result = shared_result.get() if not shared_result.empty() else False
            call_count = shared_call_counter.get() if not shared_call_counter.empty() else 0

            dpll_counter.append(call_count)
            result_list.append(result)


            if(formula_counter % verbose == 0):
                if result:
                    satisfiable_count += 1
                    print('Formula {}: SAT'.format(formula_counter))
                else:
                    if (termination_check == False):
                        print('Formula {}: UNSAT'.format(formula_counter))
                    else:
                        unsolved_counter += 1
                        print('Formula {}: UNSOLVED'.format(formula_counter))

        results[ratio] = {'time_taken': run_time,
                        'sat_count': satisfiable_count,
                        'dpll_calls': dpll_counter,
                        'sat_list': result_list,
                        'unsolved': unsolved_counter,
                        'num_formula': formula_counter - 1}
        print('------------------------------------------\n')
        
        print('Results: \n')
        if((formula_counter - 1 - unsolved_counter) == 0):
            print('Satisfiability Ratio:        Every Forumula is unsolved')
        else:
            print('Satisfiability Ratio:        {:0}'.format(results[ratio]['sat_count']/(formula_counter - 1 - unsolved_counter)))
        print('Solvability Ratio:           {:0}'.format((results[ratio]['num_formula'] - results[ratio]['unsolved'])/results[ratio]['num_formula']))
        print('Median Time Taken:           {:0.5f} secs'.format(statistics.median(results[ratio]['time_taken'])))
        print('Median No. of DPLL Calls:    {:0}\n'.format(statistics.median(results[ratio]['dpll_calls'])))
        print('==========================================\n')
    
    sys.stdout = original_stdout

with open('results_rh.txt', 'w') as file:
    file.write(json.dumps(results, indent=4))

## Two Clause Heurisitic

In [6]:
# Function to perform Pure Literal Elimination
def pure_literal_elimination(clauses):
    assignment = {}  # Dictionary to store variable assignments
    # Flatten the list of clauses to get all literals
    literals = [lit for clause in clauses for lit in clause]
  
    # Loop through all literals to find pure literals and assign them
    for literal in literals:
        if literal.startswith('¬'):
            pos_literal = literal[1:]
            # If the positive form of the literal is not in literals, it's a pure literal
            if pos_literal not in literals:
                assignment[pos_literal] = False  # Assign False to the variable
        else:
            neg_literal = '¬' + literal
            # If the negative form of the literal is not in literals, it's a pure literal
            if neg_literal not in literals:
                assignment[literal] = True  # Assign True to the variable

    return assignment

# Function to perform Unit Propagation
def unit_propagation(clauses):
    assignment = {}  # Dictionary to store variable assignments
    # Find unit clauses (clauses with only one literal)
    unit_clauses = [clause for clause in clauses if len(clause) == 1]

    # Assign values to make all unit clauses true
    for clause in unit_clauses:
        literal = clause[0]
        if literal.startswith('¬'):
            assignment[literal[1:]] = False  # Negate the literal and assign False
        else:
            assignment[literal] = True  # Assign True to the literal

    return assignment

# Function to apply the current assignment to clauses
def apply_assignment(clauses, assignment):
    updated_clauses = []  # List to store updated clauses

    # Loop through all clauses
    for clause in clauses:
        new_clause = []  # List to store literals that are not yet assigned
        for literal in clause:
            var = literal[1:] if literal.startswith('¬') else literal  # Remove the negation symbol if it exists
            if var in assignment:
                # If the literal makes the clause true, skip adding the clause
                if (literal.startswith('¬') and not assignment[var]) or \
                   (not literal.startswith('¬') and assignment[var]):
                    new_clause = None
                    break
            else:
                new_clause.append(literal)  # Add unassigned literals to new clause
        
        if new_clause is not None:
            updated_clauses.append(new_clause)  # Add the updated clause to the list of clauses

    return updated_clauses

tried_literals = []  # Stack to keep track of literals we've tried


def two_clause_heuristic(clauses):
    for clause in clauses:
        if len(clause) == 2:
            return clause[0]
    return None

def dpll_tch(clauses, assignment):
    # Apply unit propagation and pure literal elimination to simplify clauses and assignment
    new_assignment = {**unit_propagation(clauses), **pure_literal_elimination(clauses)}
    if new_assignment:
        assignment.update(new_assignment)
        clauses = apply_assignment(clauses, assignment)

    # If there's an empty clause, the formula is unsatisfiable
    if any(len(clause) == 0 for clause in clauses):
        return False, {}

    # If there are no more clauses, the formula is satisfiable with the current assignment
    if not clauses:
        return True, assignment

    # Use the two-clause heuristic to select a variable
    literal = two_clause_heuristic(clauses)

    # If the two-clause heuristic didn't find anything, just pick the first literal from the first clause
    if not literal:
        literal = clauses[0][0]

    # Convert the literal to a variable name
    var = literal[1:] if literal.startswith('¬') else literal

    global call_counter_tch
    call_counter_tch += 1

    # Try to assign True to the chosen variable and solve recursively
    new_clauses = apply_assignment(clauses, {var: not literal.startswith('¬')})
    result, new_assignment = dpll_tch(new_clauses, {**assignment, var: not literal.startswith('¬')})
    if result:
        return True, new_assignment

    # Try to assign False to the chosen variable and solve recursively
    new_clauses = apply_assignment(clauses, {var: literal.startswith('¬')})
    result, new_assignment = dpll_tch(new_clauses, {**assignment, var: literal.startswith('¬')})
    if result:
        return True, new_assignment

    # If neither True nor False leads to a solution, the formula is unsatisfiable
    return False, {}



### Solver Cell

In [7]:
results = {}
TIME_THRESHOLD = 360  # seconds

original_stdout = sys.stdout 

def run_dpll(clauses, shared_result, shared_call_counter):
    global call_counter_tch
    call_counter_tch = 0
    result, assignment = dpll_tch(clauses, {})
    shared_result.put(result)
    shared_call_counter.put(call_counter_tch)

def print_elapsed_time(start_time, end_signal):
    while not end_signal.is_set():
        elapsed_time = time.time() - start_time.value
        print(f"Time elapsed: {elapsed_time:.2f} seconds")
        time.sleep(1)

with open('output_log_tch.txt', 'w') as f:
    sys.stdout = f

    for ratio, formula_list in formulas.items():

        print('Verifying for Two Clause Heuristic - Ratio: {}\n'.format (round(ratio, 1)))
        print('No of variables (N):        ', N)
        print('No of clauses (L):          ', int(N * ratio))
        print('------------------------------------------\n')
        print('Logs: \n')

        satisfiable_count = 0
        run_time = []
        dpll_counter = []
        formula_counter = 0
        result_list = []
        termination_check = False
        unsolved_counter = 0

        for clauses in formula_list:

            formula_counter += 1
            termination_check = False

            if(formula_counter % verbose == 0):
                print('Checking satisfiability for formula {}...'.format(formula_counter))

            shared_result = multiprocessing.Queue()
            shared_call_counter = multiprocessing.Queue()
            start_time = multiprocessing.Value('d', time.time())
            end_signal = multiprocessing.Event()

            process = multiprocessing.Process(target=run_dpll, args=(clauses, shared_result, shared_call_counter))
            
            process.start()
            process.join(TIME_THRESHOLD)
            
            end_signal.set()  
            
            if process.is_alive():
                print("Terminating satisfiability check for formula {} due to time exceeding threshold".format(formula_counter))
                termination_check = True
                process.terminate()
                process.join()


            end_time = time.time()
            elapsed_time = end_time - start_time.value
            run_time.append(elapsed_time)
            
            result = shared_result.get() if not shared_result.empty() else False
            call_count = shared_call_counter.get() if not shared_call_counter.empty() else 0

            dpll_counter.append(call_count)
            result_list.append(result)


            if(formula_counter % verbose == 0):
                if result:
                    satisfiable_count += 1
                    print('Formula {}: SAT'.format(formula_counter))
                else:
                    if (termination_check == False):
                        print('Formula {}: UNSAT'.format(formula_counter))
                    else:
                        unsolved_counter += 1
                        print('Formula {}: UNSOLVED'.format(formula_counter))

        results[ratio] = {'time_taken': run_time,
                        'sat_count': satisfiable_count,
                        'dpll_calls': dpll_counter,
                        'sat_list': result_list,
                        'unsolved': unsolved_counter,
                        'num_formula': formula_counter - 1}
        print('------------------------------------------\n')
        print('Results: \n')
        print('Satisfiability Ratio:        {:0}'.format(results[ratio]['sat_count']/(formula_counter - 1 - unsolved_counter)))
        print('Solvability Ratio:           {:0}'.format((results[ratio]['num_formula'] - results[ratio]['unsolved'])/results[ratio]['num_formula']))
        print('Median Time Taken:           {:0.5f} secs'.format(statistics.median(results[ratio]['time_taken'])))
        print('Median No. of DPLL Calls:    {:0}\n'.format(statistics.median(results[ratio]['dpll_calls'])))
        print('==========================================\n')
    
    sys.stdout = original_stdout

with open('results_tch.txt', 'w') as file:
    file.write(json.dumps(results, indent=4))

## Jeroslow - Wang Heurisitic - 150 variables

In [8]:
# # Function to perform Pure Literal Elimination

def pure_literal_elimination(clauses):
    assignment = {}  # Dictionary to store variable assignments
    # Flatten the list of clauses to get all literals
    literals = [lit for clause in clauses for lit in clause]
  
    # Loop through all literals to find pure literals and assign them
    for literal in literals:
        if literal.startswith('¬'):
            pos_literal = literal[1:]
            # If the positive form of the literal is not in literals, it's a pure literal
            if pos_literal not in literals:
                assignment[pos_literal] = False  # Assign False to the variable
        else:
            neg_literal = '¬' + literal
            # If the negative form of the literal is not in literals, it's a pure literal
            if neg_literal not in literals:
                assignment[literal] = True  # Assign True to the variable

    return assignment

# Function to perform Unit Propagation
def unit_propagation(clauses):
    assignment = {}  # Dictionary to store variable assignments
    # Find unit clauses (clauses with only one literal)
    unit_clauses = [clause for clause in clauses if len(clause) == 1]

    # Assign values to make all unit clauses true
    for clause in unit_clauses:
        literal = clause[0]
        if literal.startswith('¬'):
            assignment[literal[1:]] = False  # Negate the literal and assign False
        else:
            assignment[literal] = True  # Assign True to the literal

    return assignment

# Function to apply the current assignment to clauses
def apply_assignment(clauses, assignment):
    updated_clauses = []  # List to store updated clauses

    # Loop through all clauses
    for clause in clauses:
        new_clause = []  # List to store literals that are not yet assigned
        for literal in clause:
            var = literal[1:] if literal.startswith('¬') else literal  # Remove the negation symbol if it exists
            if var in assignment:
                # If the literal makes the clause true, skip adding the clause
                if (literal.startswith('¬') and not assignment[var]) or \
                   (not literal.startswith('¬') and assignment[var]):
                    new_clause = None
                    break
            else:
                new_clause.append(literal)  # Add unassigned literals to new clause
        
        if new_clause is not None:
            updated_clauses.append(new_clause)  # Add the updated clause to the list of clauses

    return updated_clauses

# Initialization of activity scores
def jeroslow_wang_heuristic(clauses):
    scores = {}  # To store the score of each literal
    
    # Calculate the score for each literal based on the clauses it appears in
    for clause in clauses:
        score_value = 2 ** -len(clause)
        for literal in clause:
            scores[literal] = scores.get(literal, 0) + score_value

    # Return the literal with the highest score
    return max(scores, key=scores.get, default=None)

def dpll_jwh(clauses, assignment={}):
    current_assignment = assignment.copy()

    # Apply unit propagation and pure literal elimination to simplify clauses and assignment
    new_assignment = {**unit_propagation(clauses), **pure_literal_elimination(clauses)}
    if new_assignment:
        current_assignment.update(new_assignment)
        clauses = apply_assignment(clauses, current_assignment)

    # If there is an empty clause, the formula is unsatisfiable
    if any(len(clause) == 0 for clause in clauses):
        return False, {}

    # If there are no more clauses, the formula is satisfiable with the current assignment
    if not clauses:
        return True, current_assignment

    # Choose an unassigned variable using the Jeroslow-Wang heuristic
    chosen_literal = jeroslow_wang_heuristic(clauses)

    if chosen_literal is None:  # If no unassigned literals are left
        return True, assignment

    # If the chosen literal is positive, attempt to assign it to True, otherwise assign to False
    if chosen_literal.startswith('¬'):
        chosen_var = chosen_literal[1:]
        new_assignment = {chosen_var: False}
    else:
        chosen_var = chosen_literal
        new_assignment = {chosen_var: True}

    global call_counter_jwh
    call_counter_jwh += 1

    # Try to assign the chosen value to the chosen variable and solve recursively
    new_clauses = apply_assignment(clauses, new_assignment)
    result, solution_assignment = dpll_jwh(new_clauses, {**current_assignment, **new_assignment})
    if result:
        return True, solution_assignment

    # If not successful, try the opposite assignment for the chosen variable
    new_assignment = {chosen_var: not new_assignment[chosen_var]}
    new_clauses = apply_assignment(clauses, new_assignment)
    result, solution_assignment = dpll_jwh(new_clauses, {**current_assignment, **new_assignment})
    if result:
        return True, solution_assignment

    # If neither assignment leads to a solution, the formula is unsatisfiable
    return False, {}

### Solver Cell

In [9]:
results = {}
TIME_THRESHOLD = 360  # seconds

original_stdout = sys.stdout 

def run_dpll(clauses, shared_result, shared_call_counter):
    global call_counter_jwh
    call_counter_jwh = 0
    result, assignment = dpll_jwh(clauses, {})
    shared_result.put(result)
    shared_call_counter.put(call_counter_jwh)

def print_elapsed_time(start_time, end_signal):
    while not end_signal.is_set():
        elapsed_time = time.time() - start_time.value
        print(f"Time elapsed: {elapsed_time:.2f} seconds")
        time.sleep(1)

with open('output_log_jwh_150.txt', 'w') as f:
    sys.stdout = f

    for ratio, formula_list in formulas.items():


        print('Verifying for Random Choice Heuristic - Ratio: {}\n'.format (round(ratio, 1)))
        print('No of variables (N):        ', N)
        print('No of clauses (L):          ', int(N * ratio))
        print('------------------------------------------\n')
        print('Logs: \n')

        satisfiable_count = 0
        run_time = []
        dpll_counter = []
        formula_counter = 0
        result_list = []
        termination_check = False
        unsolved_counter = 0

        for clauses in formula_list:

            formula_counter += 1
            termination_check = False
            

            if(formula_counter % verbose == 0):
                print('Checking satisfiability for formula {}...'.format(formula_counter))

            shared_result = multiprocessing.Queue()
            shared_call_counter = multiprocessing.Queue()
            start_time = multiprocessing.Value('d', time.time())
            end_signal = multiprocessing.Event()

            process = multiprocessing.Process(target=run_dpll, args=(clauses, shared_result, shared_call_counter))
            
            process.start()
            process.join(TIME_THRESHOLD)
            
            end_signal.set()  
            
            if process.is_alive():
                print("Terminating satisfiability check for formula {} due to time exceeding threshold".format(formula_counter))
                termination_check = True
                process.terminate()
                process.join()


            end_time = time.time()
            elapsed_time = end_time - start_time.value
            run_time.append(elapsed_time)
            
            result = shared_result.get() if not shared_result.empty() else False
            call_count = shared_call_counter.get() if not shared_call_counter.empty() else 0

            dpll_counter.append(call_count)
            result_list.append(result)


            if(formula_counter % verbose == 0):
                if result:
                    satisfiable_count += 1
                    print('Formula {}: SAT'.format(formula_counter))
                else:
                    if (termination_check == False):
                        print('Formula {}: UNSAT'.format(formula_counter))
                    else:
                        unsolved_counter += 1
                        print('Formula {}: UNSOLVED'.format(formula_counter))

        results[ratio] = {'time_taken': run_time,
                        'sat_count': satisfiable_count,
                        'dpll_calls': dpll_counter,
                        'sat_list': result_list,
                        'unsolved': unsolved_counter,
                        'num_formula': formula_counter - 1}
        print('------------------------------------------\n')
        
        print('Results: \n')
        if((formula_counter - 1 - unsolved_counter) == 0):
            print('Satisfiability Ratio:        Every Forumula is unsolved')
        else:
            print('Satisfiability Ratio:        {:0}'.format(results[ratio]['sat_count']/(formula_counter - 1 - unsolved_counter)))
        print('Solvability Ratio:           {:0}'.format((results[ratio]['num_formula'] - results[ratio]['unsolved'])/results[ratio]['num_formula']))
        print('Median Time Taken:           {:0.5f} secs'.format(statistics.median(results[ratio]['time_taken'])))
        print('Median No. of DPLL Calls:    {:0}\n'.format(statistics.median(results[ratio]['dpll_calls'])))
        print('==========================================\n')
    
    sys.stdout = original_stdout

with open('results_jwh_150.txt', 'w') as file:
    file.write(json.dumps(results, indent=4))

## Jeroslow - Wang Heurisitic - 100 variables 

### Solver Cell

In [10]:
N = 100
formulas = generate_formulas_based_on_ratios(N)
with open("formula_100.json", "w") as file:
    json.dump(formulas, file)

results = {}
TIME_THRESHOLD = 360  # seconds

original_stdout = sys.stdout 

def run_dpll(clauses, shared_result, shared_call_counter):
    global call_counter_jwh
    call_counter_jwh = 0
    result, assignment = dpll_jwh(clauses, {})
    shared_result.put(result)
    shared_call_counter.put(call_counter_jwh)

def print_elapsed_time(start_time, end_signal):
    while not end_signal.is_set():
        elapsed_time = time.time() - start_time.value
        print(f"Time elapsed: {elapsed_time:.2f} seconds")
        time.sleep(1)

with open('output_log_jwh_100.txt', 'w') as f:
    sys.stdout = f

    for ratio, formula_list in formulas.items():


        print('Verifying for Random Choice Heuristic - Ratio: {}\n'.format (round(ratio, 1)))
        print('No of variables (N):        ', N)
        print('No of clauses (L):          ', int(N * ratio))
        print('------------------------------------------\n')
        print('Logs: \n')

        satisfiable_count = 0
        run_time = []
        dpll_counter = []
        formula_counter = 0
        result_list = []
        termination_check = False
        unsolved_counter = 0

        for clauses in formula_list:

            formula_counter += 1
            termination_check = False
            

            if(formula_counter % verbose == 0):
                print('Checking satisfiability for formula {}...'.format(formula_counter))

            shared_result = multiprocessing.Queue()
            shared_call_counter = multiprocessing.Queue()
            start_time = multiprocessing.Value('d', time.time())
            end_signal = multiprocessing.Event()

            process = multiprocessing.Process(target=run_dpll, args=(clauses, shared_result, shared_call_counter))
            
            process.start()
            process.join(TIME_THRESHOLD)
            
            end_signal.set()  
            
            if process.is_alive():
                print("Terminating satisfiability check for formula {} due to time exceeding threshold".format(formula_counter))
                termination_check = True
                process.terminate()
                process.join()


            end_time = time.time()
            elapsed_time = end_time - start_time.value
            run_time.append(elapsed_time)
            
            result = shared_result.get() if not shared_result.empty() else False
            call_count = shared_call_counter.get() if not shared_call_counter.empty() else 0

            dpll_counter.append(call_count)
            result_list.append(result)


            if(formula_counter % verbose == 0):
                if result:
                    satisfiable_count += 1
                    print('Formula {}: SAT'.format(formula_counter))
                else:
                    if (termination_check == False):
                        print('Formula {}: UNSAT'.format(formula_counter))
                    else:
                        unsolved_counter += 1
                        print('Formula {}: UNSOLVED'.format(formula_counter))

        results[ratio] = {'time_taken': run_time,
                        'sat_count': satisfiable_count,
                        'dpll_calls': dpll_counter,
                        'sat_list': result_list,
                        'unsolved': unsolved_counter,
                        'num_formula': formula_counter - 1}
        print('------------------------------------------\n')
        
        print('Results: \n')
        if((formula_counter - 1 - unsolved_counter) == 0):
            print('Satisfiability Ratio:        Every Forumula is unsolved')
        else:
            print('Satisfiability Ratio:        {:0}'.format(results[ratio]['sat_count']/(formula_counter - 1 - unsolved_counter)))
        print('Solvability Ratio:           {:0}'.format((results[ratio]['num_formula'] - results[ratio]['unsolved'])/results[ratio]['num_formula']))
        print('Median Time Taken:           {:0.5f} secs'.format(statistics.median(results[ratio]['time_taken'])))
        print('Median No. of DPLL Calls:    {:0}\n'.format(statistics.median(results[ratio]['dpll_calls'])))
        print('==========================================\n')
    
    sys.stdout = original_stdout

with open('results_jwh_100.txt', 'w') as file:
    file.write(json.dumps(results, indent=4))